In [41]:
import sys
import pathlib
from IPython.display import Audio
from typing import List

import librosa
import numpy as np
import pandas as pd
from langdetect import detect
from pythainlp.tokenize import word_tokenize
import deepcut
from string import punctuation
import re
import thaispellcheck

ROOT_DIR = pathlib.Path.cwd().parent
sys.path.insert(0, str(ROOT_DIR))
from src.normalizers import EnglishTextNormalizer
from src.evaluate_utils import isd
from src.post_processer import remove_longest_repeating_words

In [42]:
target_folder = ROOT_DIR / 'data/result/Thonburian-whisper-deepbrain-lang' / 'th_Oppday_Q2_2023_IP_interfama'
label_dir = target_folder / 'Groundtruth.csv'
th_result_dir = target_folder / 'th_result.csv'
en_result_dir = target_folder / 'en_result.csv'

In [43]:
label_df = pd.read_csv(str(label_dir))
th_result_df = pd.read_csv(str(th_result_dir)) 
en_result_df = pd.read_csv(str(en_result_dir))

In [44]:
merge_result_df = pd.concat(
    [th_result_df, en_result_df],
    axis=0
)
merge_result_df.shape

(888, 3)

In [45]:
merge_result_df.to_csv('./merge_result.csv')

In [46]:
sorted_result_df = merge_result_df.sort_values('start')

In [47]:
sorted_result_df.to_csv(str(target_folder / 'en-th_result.csv'))

In [48]:
prediction = sorted_result_df['text'].to_list()
references = label_df['text'].to_list()

In [49]:
text = "Q1 "
re.sub("[QA]:","", text)

'Q1 '

In [50]:
def en_preprocess(text: str) -> str:
    text = re.sub("[QA]:|(uh)","", text)
    return text

def th_preprocess(text: str) -> str:
    text = re.sub("((ค่ะ)|((นะ)?ครับ)|(นะคะ)|เอ่อ)+", "", text)
    # text = thaispellcheck.check(text, autocorrect=True)
    return text

def pred_postprocess(listtext: List[str]) -> List[str]:

    if " " in listtext:
        listtext.remove(" ")
        
    clean_text = remove_longest_repeating_words(listtext)
    return clean_text

def tokenize_text(text: str, pred = False) -> list:
    if not isinstance(text, str):
        return []
    try:
        lang = detect(text)
    except:
        lang = 'en'
    if lang != 'en':
        text = th_preprocess(text)
        splited_text = deepcut.tokenize(text)
    else:
        eng_normalizer = EnglishTextNormalizer()
        text = eng_normalizer(text)
        text = en_preprocess(text)
        splited_text = text.split(" ")
        
    return pred_postprocess(splited_text) if pred else splited_text

In [51]:
from functools import partial
prediction_words = list(map(partial(tokenize_text, pred=True), prediction))
references_words = list(map(tokenize_text, references))

2/2 [==============================] - 0s 8ms/step
{'วัน': 2, 'นี้': 1, 'ก็': 1, 'เป็น': 1, 'Auxility': 1, ' ': 2, 'Day': 1, 'ที่': 1, 'ผม': 1, 'จะ': 1, 'รายงาน': 1}
1/1 [==============================] - 0s 37ms/step


2/2 [==============================] - 0s 6ms/step
{'ทั้ง': 1, 'ของ': 2, 'คน': 1, 'และ': 1, 'สัตว์': 1, 'จาก': 1, 'ต้น': 1, 'น้ำ': 2, ' ': 1, 'เดิน': 1, 'ตาม': 1, 'ตอน': 1}
2/2 [==============================] - 0s 5ms/step
{'ก็': 2, 'ขอ': 1, 'มอบ': 1, 'บิน': 1, 'ทรัลสุขภาพ': 1, 'แบบ': 1, 'พวก': 1, 'องค์กร': 1}
1/1 [==============================] - 0s 46ms/step
{'จะ': 1, 'พูด': 1, 'ให้': 1, 'ฟัง': 1, 'ว่า': 2, 'เป็น': 1, 'อย่าง': 1, 'ไร': 1}
2/2 [==============================] - 0s 5ms/step
{'แล้ว': 1, 'ก็': 1, 'จาก': 1, 'ส้น': 1, 'น้ำ': 2, 'เป็น': 2, 'ปลาย': 1, 'อย่าง': 1, 'ไร': 1}
1/1 [==============================] - 0s 47ms/step
{'การ': 2, 'รักษา': 1, 'แล้ว': 1, 'ก็': 1, 'ป้องกัน': 1, 'โรค': 1}
2/2 [==============================] - 0s 7ms/step
{'แล้ว': 1, 'ก็': 2, 'ถัด': 1, 'มา': 1, 'อัน': 1, 'นี้': 1, 'จะ': 1, 'เป็น': 1, 'เรื่อง': 1, 'ของ': 1, 'ยา': 1, 'ซึ่ง': 1, 'ปัจจุบัน': 1}
2/2 [==============================] - 0s 8ms/step
{'เรา': 1, 'มี': 1, 'โรง': 2, 'งาน': 1, 'ยา': 2, 

In [52]:
from functools import reduce

prediction_words_reduce = list(reduce(lambda a, b: a + b, prediction_words))
prediction_words_reduce = [word for word in prediction_words_reduce if not word in ["", " "]]
references_words_reduce = list(reduce(lambda a, b: a + b, references_words))
references_words_reduce = [word for word in references_words_reduce if not word in ["", " "]]

In [57]:
isd(prediction_words_reduce, references_words_reduce, debug=False) / len(references_words_reduce)

0.4287840452448281

In [56]:
from torchmetrics.text import CharErrorRate
cer = CharErrorRate()
cer(prediction_words_reduce, references_words_reduce,)

/home/ong_ubuntu/.cache/pypoetry/virtualenvs/set-speechtotext-poc-kSFlFoQd-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor(1.1733)